In [1]:
import re

import sqlitedatastore as datastore

ptn_relation = re.compile(
    r'(?P<cause>[^、はもがに]+[はもが])(?P<effect>[^はもが]+に)影響を与え')

def extract_relation(doc_id):
    text = datastore.get(doc_id, fl=['content'])['content']
    anno_id = 0
    for sent in datastore.get_annotation(doc_id, 'sentence'):
        for m in ptn_relation.finditer(text[sent['begin']:sent['end']]):
            relation = {
                'cause':  {'begin': m.start('cause') + sent['begin'],
                           'end':   m.end('cause')   + sent['begin'],
                           'link':  ('effect', anno_id)},
                'effect': {'begin': m.start('effect') + sent['begin'],
                           'end':   m.end('effect')   + sent['begin']},
            }
            anno_id += 1
            yield sent, relation


datastore.connect()
for doc_id in datastore.get_all_ids(limit=-1):
    text = datastore.get(doc_id, fl=['content'])['content']
    for sent, relation in extract_relation(doc_id):
        print('文書{0:d} {1:s}'.format(
            doc_id, text[sent['begin']:sent['end']]))
        for anno_name, anno in relation.items():
            print('\t{0}: {1}'.format(
                anno_name, text[anno['begin']:anno['end']]))
        print()
datastore.close()


文書3 江戸時代の日本の文化は「ジャポニズム」として印象派美術などフランス文化に影響を与えた
	cause: 江戸時代の日本の文化は
	effect: 「ジャポニズム」として印象派美術などフランス文化に

文書11 パーラ朝が仏教を保護してパハルプールの仏教寺院（現在はバングラデシュ領内）が建設され、近隣諸国のパガン仏教寺院・アンコール仏教寺院・ボロブドゥール仏教寺院の建設に影響を与えた
	cause: バングラデシュ領内）が
	effect: 建設され、近隣諸国のパガン仏教寺院・アンコール仏教寺院・ボロブドゥール仏教寺院の建設に

文書11 仏教と異なりインド以外の地にはほとんど伝わらなかったが、その国内に深く根を下ろし、およそ2500年の長い期間にわたりインド文化の諸方面に影響を与え続け、今日もなおわずかだが無視できない信徒数を保っている
	cause: ほとんど伝わらなかったが
	effect: 、その国内に深く根を下ろし、およそ2500年の長い期間にわたりインド文化の諸方面に

文書11 ゴア料理はポルトガルとその植民地に伝播し、マカオ料理などに影響を与えた
	cause: ゴア料理は
	effect: ポルトガルとその植民地に伝播し、マカオ料理などに



In [3]:
# 一回だけ実行する
# import sqlite3
# conn = sqlite3.connect('sample.db')
# conn.execute("ALTER TABLE docs ADD COLUMN '%s' 'BLOB'" % 'cause')
# conn.execute("ALTER TABLE docs ADD COLUMN '%s' 'BLOB'" % 'effect')
# conn.close()

In [4]:
import sqlitedatastore as datastore
from annoutil import find_xs_in_y


def extend_phrase(chunk, chunk_tokens, tokens, all_chunks):
    def _extend(chunk, chunk_tokens):
        for child in all_chunks:
            _, link = child['link']
            if link == -1:
                continue
            if all_chunks[link] != chunk:
                continue
            child_tokens = find_xs_in_y(tokens, child)
            if child_tokens[0]['POS'] == chunk_tokens[0]['POS']:
                return [child] + _extend(child, child_tokens)
        return []

    phrase = [chunk] + _extend(chunk, chunk_tokens)
    return {
        'begin': min(phrase, key=lambda x: x['begin'])['begin'],
        'end':   max(phrase, key=lambda x: x['end'])['end'],
    }


def find_child(parent, chunks_in_sent, tokens_in_sent, text, all_chunks, child_cond):
    for child in chunks_in_sent:
        _, link = child['link']
        if link == -1 or all_chunks[link] != parent:
            continue
        child_tokens = find_xs_in_y(tokens_in_sent, child)
        if text[child['begin']:child['end']] in child_cond.get('text', []):
            return child, child_tokens
        if child_tokens[-1]['POS'] in child_cond.get('pos1', []) and \
                child_tokens[-1]['lemma'] in child_cond.get('lemma1', []) and \
                child_tokens[-2]['POS'] not in child_cond.get('pos2_ng', []):
            return child, child_tokens
    return None, None


def extract_relation(doc_id):
    text = datastore.get(doc_id, fl=['content'])['content']
    all_chunks = datastore.get_annotation(doc_id, 'chunk')
    all_tokens = datastore.get_annotation(doc_id, 'token')
    anno_id = 0
    for sent in datastore.get_annotation(doc_id, 'sentence'):
        chunks = find_xs_in_y(all_chunks, sent)
        tokens = find_xs_in_y(all_tokens, sent)
        for chunk in chunks:
            chunk_tokens = find_xs_in_y(tokens, chunk)
            if not any([chunk_token['lemma'] == '与える'
                        for chunk_token in chunk_tokens]):
                continue

            affect, affect_tokens = find_child(
                chunk, chunks, tokens, text, all_chunks,
                child_cond={ 'text': ['影響を'] })
            if affect is None:
                continue

            cause, cause_tokens = find_child(
                chunk, chunks, tokens, text, all_chunks,
                child_cond={
                    'pos1':    ['助詞'],
                    'lemma1':  ['は', 'も', 'が'],
                    'pos2_ng': ['助詞'],
                })
            if cause is None:
                continue

            effect, effect_tokens = find_child(
                chunk, chunks, tokens, text, all_chunks,
                child_cond={
                    'pos1':    ['助詞'],
                    'lemma1':  ['に'],
                    'pos2_ng': ['助詞'],
                })
            if effect is None:
                continue
            
            cause  = extend_phrase(cause,  cause_tokens,  tokens, all_chunks)
            effect = extend_phrase(effect, effect_tokens, tokens, all_chunks)

            relation = {
                'cause':  {
                    'begin': cause['begin'],
                    'end':   cause['end'],
                    'link':  ('effect', anno_id),
                },
                'effect':  {
                    'begin': effect['begin'],
                    'end':   effect['end'],
                }
            }

            anno_id += 1
            yield sent, relation



datastore.connect()
for doc_id in datastore.get_all_ids(limit=-1):
    text = datastore.get(doc_id, fl=['content'])['content']
    annotations = {}
    for sent, relation in extract_relation(doc_id):
        print('文書{0:d} {1}'.format(doc_id, text[sent['begin']:sent['end']]))
        for anno_name, anno in relation.items():
            print('\t{0}: {1}'.format(
                anno_name, text[anno['begin']:anno['end']]))
            annotations.setdefault(anno_name, []).append(anno)
        print()
    for anno_name, annos in annotations.items():
        datastore.set_annotation(doc_id, anno_name, annos)
datastore.close()


文書2 このテネンチズモ（ポルトガル語版）は直接は国政に大きな影響を与えなかったが、間接的に1930年代の政治状況を用意することになった
	cause: （ポルトガル語版）は
	effect: 国政に

文書3 江戸時代の日本の文化は「ジャポニズム」として印象派美術などフランス文化に影響を与えた
	cause: 江戸時代の日本の文化は
	effect: 印象派美術などフランス文化に

文書6 19世紀にはラルフ・ワルド・エマーソンや隠遁者ヘンリー・デイヴィッド・ソロー、ウォルト・ホイットマンらの超越論哲学と、チャールズ・サンダース・パース、ウィリアム・ジェームズ、ジョン・デューイらのプラグマティズム哲学がおもな潮流となり、特にウィリアム・ジェームズの『純粋経験論』は日本の西田幾多郎の初期西田哲学（『善の研究』）に大きな影響を与えている
	cause: 『純粋経験論』は
	effect: 日本の西田幾多郎の初期西田哲学（『善の研究』）に

文書12 同じころ、オーギュスト・コントは実証主義を唱え、実証主義は19世紀後半のラテンアメリカ諸国の政治や文化（1889年のブラジルの共和制革命など）に大きな影響を与えた
	cause: 実証主義は
	effect: 19世紀後半のラテンアメリカ諸国の政治や文化（1889年のブラジルの共和制革命など）に



In [5]:
import json

import sqlitedatastore as datastore
import solrindexer     as indexer
from annoutil import find_x_including_y


def create_index_data(doc_id, meta_info, anno_name, anno, i, sent, text):
    ref_anno_name, link = anno['link']
    ref_anno = datastore.get_annotation(doc_id, ref_anno_name)[link]
    data = {
        'id':                       '{0:d}.{1:s}.{2:d}'.format(doc_id, anno_name, i),
        'doc_id_i':                 doc_id,
        'anno_id_i':                i,
        'name_s':                   anno_name,
        'sentence_txt_ja':          text[sent['begin']:sent['end']],
        anno_name + '_txt_ja':      text[anno['begin']:anno['end']],
        ref_anno_name + '_txt_ja':  text[ref_anno['begin']:ref_anno['end']],
        'title_txt_ja':             meta_info['title'],
        'url_s':                    meta_info['url'],
    }
    return data



datastore.connect()
anno_name = 'cause'
data = []
for doc_id in datastore.get_all_ids(limit=-1):
    row = datastore.get(doc_id, fl=['content', 'meta_info'])
    text = row['content']
    meta_info = json.loads(row['meta_info'])
    sents = datastore.get_annotation(doc_id, 'sentence')
    for i, anno in enumerate(datastore.get_annotation(doc_id, anno_name)):
        sent = find_x_including_y(sents, anno)
        data.append(create_index_data(doc_id, meta_info,
            anno_name, anno, i, sent, text))

# Solr への登録を実行
indexer.load('anno', data)
datastore.close()


{
  "responseHeader":{
    "status":0,
    "QTime":394}}

{
  "responseHeader":{
    "status":0,
    "QTime":188}}



In [8]:
import json

import solrindexer as indexer


results = indexer.search_annotation(
    fl_keyword_pairs=[
        ('cause_txt_ja', [['江戸']]),
        ('name_s',       [['cause']])
    ])
print(json.dumps(results, indent=4, ensure_ascii=False))


{
    "responseHeader": {
        "status": 0,
        "QTime": 4,
        "params": {
            "q": "(cause_txt_ja:\"江戸\") AND (name_s:\"cause\")",
            "rows": "100",
            "wt": "json"
        }
    },
    "response": {
        "numFound": 1,
        "start": 0,
        "docs": [
            {
                "id": "3.cause.0",
                "doc_id_i": 3,
                "anno_id_i": 0,
                "name_s": "cause",
                "sentence_txt_ja": "江戸時代の日本の文化は「ジャポニズム」として印象派美術などフランス文化に影響を与えた",
                "cause_txt_ja": "江戸時代の日本の文化は",
                "effect_txt_ja": "印象派美術などフランス文化に",
                "title_txt_ja": "日本",
                "url_s": "https://ja.wikipedia.org/wiki/%E6%97%A5%E6%9C%AC",
                "_version_": 1666725182141628416
            }
        ]
    }
}
